# Visualization for Core-level spectra

Welcome! Here you can plot theoretical core-level spectra from the data in the database.
This GUI just wraps some functionalities of the method 'plot_methods.plot_fleur_aiida.plot_spectra'.
You can export the data or save the plot. Enjoy! Created @ PGI-1 by Jens Broeder 

In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [ ]:
from __future__ import print_function

from aiida import load_profile
load_profile()

from aiida.orm import FolderData
from aiida.orm import load_node 

import ipywidgets as ipw
from IPython.display import display, clear_output
import nglview

from urllib.parse import urlparse, parse_qs, urlsplit
from collections import OrderedDict
from pprint import pformat
import ase.io
from base64 import b64encode

#from plot_methods.plot_fleur_aiida import plot_spectra
#from aiida_fleur.tools.io_routines import write_xps_spectra_datafile

#import plot_methods

In [ ]:
from xps_analysis.plot.plot_spectrum import plot_spectra
from aiida_fleur.tools.io_routines import write_xps_spectra_datafile


In [ ]:
#?plot_spectra

In [ ]:
def prepare_input():
    inputs = {}
    inputs['energy_range'] = [0, 200]
    #node = load_node(int(inp_pk.value))
    pklist = inp_pks.value.strip().split()
    if pklist:
        pklist_new = []
        for pk in pklist:
            #try:
            pk_new = int(pk)
            #except:
            pklist_new.append(pk_new) 
        inputs['wc_nodes'] = pklist_new
    
    if inp_egrid.value:
        inputs['energy_grid'] = inp_egrid.value
    
    if inp_erange_min.value:
        inputs['energy_range'] = [inp_erange_min.value, inputs['energy_range'][1]]

    if inp_erange_max.value:
        inputs['energy_range'] = [inputs['energy_range'][0], inp_erange_max.value]

    if inp_fwhm_l.value:
        inputs['fwhm_l'] = inp_fwhm_l.value

    if inp_fwhm_g.value:
        inputs['fwhm_g'] = inp_fwhm_g.value

    peakfunc = inp_peakfun.value.strip().split()
    if peakfunc:        
        if len(peakfunc) != 1:
            print('You can only provide one peakfunction name. I use voigts as default...')
        else:
            inputs['peakfunction'] = peakfunc[0]
    
    factors = inp_factors.value.strip().split()
    if factors:        
        if len(factors) != len(pklist):
            print('The number of factors you provide must be equals to the number of nodes provided. I skip these factors...')
        else:
            factors_new = []
            for factor in factors:
                factors_new.append(float(factor))
            inputs['factors'] = factors_new
            print(factors_new)
        
    return inputs

#plot_spectra(wc_nodes, title='', factors=[], energy_range=[100, 120], fwhm_g=0.6, fwhm_l=0.1, energy_grid=0.2, peakfunction='voigt', linetyp_spec='o-', warn_ref=False, **kwargs)

In [ ]:
def prepare_download_input():
    return_dict = {}
 
    pklist = inp_pks.value.strip().split()
    if pklist:
        pklist_new = []
        for pk in pklist:
            #try:
            pk_new = int(pk)
            #except:
            pklist_new.append(pk_new) 
        return_dict['nodes'] = pklist_new

    factors = inp_factors.value.strip().split()
    if factors:        
        if len(factors) != len(pklist):
             print('The number of factors you provide must be equals to the number of nodes provided. I skip these factors...')
        else:
            factors_new = []
            for factor in factors:
                factors_new.append(float(factor))
            return_dict['factors'] = factors_new
            #print(factors_new)
    else:
        return_dict['factors'] = []
    
    return_dict['destination'] = './outdata.txt'


    
    return return_dict
#return all_wc_BeTi_uuid, factors, *returnvalues, destination=destination)

In [ ]:
# search UI
style = {"description_width":"100px"}
layout = ipw.Layout(width="692px")
btn_sub = ipw.Button(description="Plot")
btn_download = ipw.Button(description="Download Data")
inp_pks = ipw.Text(description="Nodes PK/UUID: ", placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)
inp_factors = ipw.Text(description="Factors:", placeholder='e.g. 5.1 1 1 (space seperator # of factors has to be equal # of pks)', layout=layout, style=style)
inp_erange_min = ipw.BoundedFloatText(value=0, min=0, max=2000, step=0.1, description='Emin [eV]:', disabled=False, 
                            continuous_update=False, readout_format='.1f')
inp_erange_max = ipw.BoundedFloatText(value=400, min=0, max=2000, step=0.1, description='Emax [eV]:', disabled=False, 
                            continuous_update=False,  readout_format='.1f')
inp_egrid = ipw.FloatSlider(value=0.020, min=0, max=0.5, step=0.001, description='Energy grid [eV]:', disabled=False, 
                            continuous_update=False, orientation='horizontal', readout=True, readout_format='.3f')
inp_fwhm_l = ipw.FloatSlider(value=0.1, min=0, max=2.0, step=0.01, description='fwhm_l [eV]:', disabled=False, 
                            continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
inp_fwhm_g = ipw.FloatSlider(value=0.6, min=0, max=2.0, step=0.01, description='fwhm_g [eV]:', disabled=False, 
                            continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
inp_peakfun = ipw.Text(description="Peakfunction type:", placeholder='e.g. voigt gaus lorentz pseudo-voigt (only one)', layout=layout, style=style)




function_user_input = [inp_pks, inp_factors, inp_erange_min, inp_erange_max, inp_egrid, inp_fwhm_l, inp_fwhm_g, inp_peakfun]

In [ ]:
# disable scolling

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
data_global = None
def on_submit(b):
    global stack_list
    global data_global
    with output:
        clear_output()
        function_input = prepare_input()
        data = plot_spectra(**function_input)
        #print(data)
        data_global = data
        
def download_data(b):
    global stack_list
    if not data_global:
        print('HINT: You first have to plot some data, before exporting.')
        return
    download_input = prepare_download_input()
    destination = download_input['destination']
    #print('Storing datafile under {}'.format(destination))
    write_xps_spectra_datafile(download_input['nodes'], download_input['factors'], *data_global, destination=destination)
        
inp_pks.on_submit(on_submit)
btn_sub.on_click(on_submit)
btn_download.on_click(download_data)
output = ipw.Output()
app = ipw.VBox(children=function_user_input + [btn_download, btn_sub, output])
display(app)

In [ ]:
#stack_list = ipw.VBox()
#display(stack_list)

In [ ]:
# parse URL
url = urlsplit(jupyter_notebook_url)
querypart = parse_qs(url.query)
if 'pk' in querypart.keys():
    #print(querypart['pk'])
    pksting = ''
    for pk in querypart['pk']:
        pksting = pksting + '{} '.format(pk)
    inp_pks.value = pksting.strip()
    on_submit(None)

In [ ]:
#!ls

In [ ]:
#!cat out.txt